In [53]:
import pandas as pd
import numpy as np

In [65]:
baseinfo = pd.read_excel("data.xlsx", sheet_name="baseinfo", index_col=0); baseinfo

,name,group
code,,
A005930,삼성전자,반도체및하드웨어
A373220,LG에너지솔루션,반도체및하드웨어
A000660,SK하이닉스,반도체및하드웨어
A207940,삼성바이오로직스,의료
A051910,LG화학,화학
...,...,...
A003850,보령,의료
A192820,코스맥스,생활용품
A280360,롯데제과,음식료및담배


# PER

In [109]:
data_per = pd.read_excel("data.xlsx", sheet_name="Raw(PER)", skiprows=[0,1,2,3,4,5,6,7,9,10,11,12,13], index_col=0)
data_per = data_per.iloc[0]
data_per = data_per.fillna(10)
data_per.index.name = 'code'
data_per

code
A005930    20.855987
A373220    73.103610
A000660    10.000000
A207940    66.081463
A051910    20.763204
             ...    
A003850    15.698010
A192820    11.507434
A280360     9.978186
A057050     4.154491
A284740    10.000000
Name: 2023-03-02 00:00:00, Length: 200, dtype: float64

In [110]:
pd.merge(baseinfo['group'], data_per, on='code').groupby('group').mean()

,2023-03-02 00:00:00
group,
건설,6.262589
금속및광물,8.471856
기타 소재,14.811308
기타자본재,9.309688
내구소비재및의류,14.263202
미디어,23.686435
반도체및하드웨어,23.221026
보험및기타금융,32.053140
상업서비스,20.116120


In [111]:
z_per = pd.merge(baseinfo['group'], data_per, on='code').groupby('group').transform(lambda x: -(x - x.mean()) / x.std())
z_per

,2023-03-02 00:00:00
code,
A005930,0.100771
A373220,-2.125436
A000660,0.563332
A207940,-2.607436
A051910,-0.516153
...,...
A003850,0.332868
A192820,0.971641
A280360,0.334339


In [112]:
z_per.sum()

2023-03-02    3.552714e-15
dtype: float64

# PBR

In [66]:
data_pbr = pd.read_excel("data.xlsx", sheet_name="Raw(PBR)", skiprows=[0,1,2,3,4,5,6,7,9,10,11,12,13], index_col=0)
data_pbr = data_pbr.iloc[0]
data_pbr = data_pbr.fillna(10)
data_pbr.index.name = 'code'
data_pbr

code
A005930     1.19644
A373220    13.43189
A000660     0.99500
A207940    10.40599
A051910     2.44451
             ...   
A003850     1.23309
A192820     1.73392
A280360     0.61674
A057050     0.31063
A284740     1.02439
Name: 2023-03-02 00:00:00, Length: 200, dtype: float64

In [68]:
pd.merge(baseinfo['group'], data_pbr, on='code').groupby('group').mean()

,2023-03-02 00:00:00
group,
건설,0.778710
금속및광물,0.690393
기타 소재,1.724005
기타자본재,0.910273
내구소비재및의류,1.893445
미디어,2.202995
반도체및하드웨어,2.889103
보험및기타금융,2.015451
상업서비스,1.912820


In [69]:
z_pbr = pd.merge(baseinfo['group'], data_pbr, on='code').groupby('group').transform(lambda x: -(x - x.mean()) / x.std())
z_pbr

,2023-03-02 00:00:00
code,
A005930,0.469729
A373220,-2.925718
A000660,0.525631
A207940,-2.160121
A051910,-0.272698
...,...
A003850,0.710577
A192820,-0.038631
A280360,0.839918


# Rank

In [113]:
rank = (0.5*z_per + 0.5*z_pbr).rank(ascending=False).squeeze()
rank = pd.merge(baseinfo['name'], rank.sort_values(), on='code')
rank.columns = ['name', 'rank']
rank.sort_values(by='rank').head(10)

,name,rank
code,,
A009540,한국조선해양,1.0
A001680,대상,2.0
A000210,DL,3.0
A000880,한화,4.0
A097950,CJ제일제당,5.0
A004020,현대제철,6.0
A000120,CJ대한통운,7.0
A057050,현대홈쇼핑,8.0
A069960,현대백화점,9.0


# Pickling

In [115]:
baseinfo.to_pickle('baseinfo.pkl')

In [119]:
per_hist = pd.read_excel("data.xlsx", sheet_name="Raw(PER)", skiprows=[0,1,2,3,4,5,6,7,9,10,11,12,13], index_col=0)
per_hist.index.name = 'code'
per_hist.to_pickle('per_hist.pkl')

In [120]:
pbr_hist = pd.read_excel("data.xlsx", sheet_name="Raw(PBR)", skiprows=[0,1,2,3,4,5,6,7,9,10,11,12,13], index_col=0)
pbr_hist.index.name = 'code'
pbr_hist.to_pickle('pbr_hist.pkl')